In [18]:
import pandas as pd
import xmlrpc.client

In [19]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)



pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")

# Merge claficacion

test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard" , "display_name":"centro_produccion"})

pareto_prod = pareto_prod[["nom_produccion","Clasificacion"]] #Columna realmente requeridas
te_clas = test.merge(pareto_prod, how="left",on="nom_produccion") 

# Descartar productos sin ventas ya que no tienen clasificacion
sin_ventas = te_clas[te_clas["Clasificacion"].isna()] 
te_clas = te_clas[~te_clas["nom_produccion"].isin(sin_ventas["nom_produccion"])]

te_clas = te_clas[["nom_produccion","producto","Clasificacion","centro_produccion","tiempo_estimado_estandard"]]


In [20]:
te_clas[te_clas["nom_produccion"]=="DA2547"]

,nom_produccion,producto,Clasificacion,centro_produccion,tiempo_estimado_estandard
787,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,ENTUBADORA 3,50.0
1202,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,ENTUBADORA 2,40.0
1704,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,PLISADORA 3,0.0
2025,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,PLISADORA 4,170.0
2420,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,HENNECKE,225.0
3424,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,BOLSA,0.0
3667,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,MARCADO/HOTMELT/ARANDELA/EMPAQUE,40.0
4697,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,CONTROL DE CALIDAD,30.0
4953,DA2547,"DA2547 F.AIRE 1_ DONSSON -KODIAK, FORD 7000 C...",AAA,EMBALAJE,45.0


In [21]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# 1) Traer Workorders (órdenes de trabajo de producción)
# ===============================

workorders = models.execute_kw(
    db, uid, password,
    "mrp.production.workcenter.line", "search_read",
    [
        [["date_start", ">=", "2023-01-01"], ["state", "=", "done"]],  # domain
        ["id", "name", "production_id", "product_id", "product_internal_ref",
         "workcenter_id", "date_start", "date_finished", "hour",
         "tiempo_optimo_horas", "tiempo_estimado_segundos",
         "tiempo_segundos_hombre","qty","user_start","fuerza_laboral"]
    ]  # <-- fields como lista, no dict
)

import pandas as pd
df_workorders = pd.DataFrame(workorders)

df_workorders["OP"] = df_workorders["production_id"].apply(
    lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)

df_workorders["operario"] = df_workorders["user_start"].apply(
    lambda x: x[1]   if isinstance(x,list) and len(x) >1 else None
)


df_workorders.loc[:, "centro_produccion"] = df_workorders["name"].str.split(" - ", n=1).str[0].str.strip()

df_workorders.loc[:, "nom_produccion"] = df_workorders["name"].str.extract(r'([A-Z]{2}\d{4})')


In [22]:
df_ops = df_workorders[["nom_produccion","OP","date_finished","centro_produccion","operario","qty","tiempo_segundos_hombre"]]
df_ops.sample(10)

,nom_produccion,OP,date_finished,centro_produccion,operario,qty,tiempo_segundos_hombre
30177,DA3309,MO09784,2023-11-07 13:25:48,CONTROL DE CALIDAD,INGRID CAROLINA GOMEZ DELPRADO,20.0,821.60
8013,DA2033,MO12613,2025-06-14 12:02:21,ENTUBADORA 2,FRANCY CAROLINA PINZON CHACON,30.0,1324.73
27359,DA4653,MO10859,2024-07-16 11:35:18,CONTROL DE CALIDAD,MARIA HELENA DELPRADO PEREZ,30.0,1315.20
21727,DA2270,MO09147,2023-06-16 10:41:15,PLISADORA,DAVID ESTEBAN MALAGON CETINA,50.0,30403.46
11357,DA2690,MO09497,2023-08-30 15:05:11,PLISADORA,BLANCA CECILIA AVELLA ARIAS,500.0,111292.50
25269,DA3066,MO08416,2023-01-12 13:53:19,INYECCION,ALIPTER GUARACA MONTEALEGRE,300.0,73218.50
33774,DA2982,MO11967,2025-02-22 10:53:36,CONTROL DE CALIDAD,MARIA HELENA DELPRADO PEREZ,600.0,27396.60
26252,DA4636,MO09478,2023-09-06 06:57:33,CONTROL DE CALIDAD,INGRID CAROLINA GOMEZ DELPRADO,30.0,3701.08
27608,DA2109,MO11130,2024-08-27 14:23:22,INYECCION,ALIPTER GUARACA MONTEALEGRE,100.0,19005.45
13851,DA4946,MO08509,2023-02-14 14:37:00,INYECCION MESA INTERNOS,LUISA FERNANDA PARRA REYES,200.0,93859.43


In [23]:
tr_clas =df_ops.merge(pareto_prod, how="left",on="nom_produccion") 

In [33]:
df_def = tr_clas.merge(te_clas, how="left", on=("nom_produccion","centro_produccion","Clasificacion"))

In [41]:
df_def[df_def["Clasificacion"]=="AAA"].sample(10)


,nom_produccion,OP,date_finished,centro_produccion,operario,qty,tiempo_segundos_hombre,Clasificacion,producto,tiempo_estimado_estandard
25299,DA4772,MO12931,2025-08-08 08:44:57,BOLSA,INGRID CAROLINA GOMEZ DELPRADO,500.0,0.00,AAA,NaN,NaN
32095,DA4570,MO12334,2025-05-14 09:43:53,EMBALAJE,CLARA MARLEN OSPINA CASTRO,850.0,16405.00,AAA,"DA4570 F. AIRE 2. DONSSON - PERKINS, CATERPIL...",30.0
6710,DA2772,MO10349,2024-03-14 13:36:21,ENTUBADORA 2,FRANCY CAROLINA PINZON CHACON,800.0,16491.20,AAA,"DA2772 F. AIRE BOBCAT, HITACHI (CARCASA GX2772A)",35.0
9897,DA4690,MO11318,2024-09-25 14:11:47,PLISADORA 1,BLANCA CECILIA AVELLA ARIAS,400.0,68709.14,AAA,NaN,NaN
24216,DA2772,MO11689,2024-12-18 15:43:34,CONTROL DE CALIDAD,OLGA YANETH TORRES TRIANA,800.0,32844.00,AAA,"DA2772 F. AIRE BOBCAT, HITACHI (CARCASA GX2772A)",30.0
1378,DA2957,MO08446,2023-01-30 14:48:57,ENTUBADORA 1,JAMER PUPO SIERRA,400.0,19701.33,AAA,NaN,NaN
21758,DA4982,MO08921,2023-05-16 13:07:48,MARCADO/HOTMELT/ARANDELA/EMPAQUE,ERIKA YULIEDT UMAÑA SABOGAL,300.0,22070.00,AAA,DA4982 F. AIRE INT. CAMION IHC 7600 MOTOR CUMMINS,50.0
11489,DA8168,MO09518,2023-08-31 15:40:40,ENTUBADORA 2,YENNY KATERINE PATAQUIVA NEUSA,800.0,24638.22,AAA,DA8168 F. AIRE CAMION FOTON AUMARK BJ1065 M4 E...,50.0
17515,DA2982,MO08611,2023-02-23 10:20:05,PLISADORA,EDINSON PUPO SIERRA,500.0,154961.88,AAA,NaN,NaN
22364,DA2671,MO09707,2023-10-26 07:58:22,INYECCION,JAVIER ESTEBAN GONZALEZ ARENAS,800.0,111398.00,AAA,NaN,NaN
